In [1]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm
from spacy.lang.fr import French
import pandas as pd
import json
import ast
from spacy_langdetect import LanguageDetector
import minibatch
import random
from spacy.util import minibatch, compounding
from spacy.training import Example
import numpy as np

In [2]:
nlp = French()

### Datas

In [10]:
def read_jsonl(path_json_file):
    with open(path_json_file, 'r') as json_file:
        json_list = list(json_file)
    return json_list

path_json_file = "/Users/remyadda/Desktop/AD/Projets/ALFRED/Datas/Doccano/doccano_skills_outputs_360.jsonl"
DATA = read_jsonl(path_json_file)

In [11]:
DATA[0]

'{"id": 364, "text": "LOGICIELS MAITRISES: WORD -- EXCEL -- SAGE -- POWERPOINT -- OUTLOOK", "meta": {}, "annotation_approver": null, "labels": [[21, 25, "U-SKILLS"], [29, 34, "U-SKILLS"], [38, 42, "U-SKILLS"], [46, 56, "U-SKILLS"], [60, 67, "U-SKILLS"]]}\n'

In [12]:
def reformate_datas(TRAIN_DATA):
    """Input: JSON file issu de Doccano
       Output: Datas mis au format du NER"""
    TRAIN = []
    for element in TRAIN_DATA:
        # null pose pb. On le remplace par un mot qqconque
        element = element.replace('null', '"ok"')
        d = eval(element)
        dictOfLabels = {}
        dictOfLabels['entities'] = d['labels']
        listOfTuples = []
        for element in dictOfLabels['entities']:
            element = tuple(element)
            listOfTuples.append(element)
        dictOfLabels['entities'] =  listOfTuples
        TRAIN.append((d['text'], dictOfLabels))
    return TRAIN

### Reformate + Train Test Split

In [13]:
DATA = reformate_datas(DATA)
print(len(DATA))

361


In [14]:
TRAIN = DATA[:int(0.8*len(DATA))]
TEST = DATA[int(0.8*len(DATA)):]
print('Len train set : ', len(TRAIN))
print('Len test set : ', len(TEST))

Len train set :  288
Len test set :  73


In [20]:
TRAIN

[('LOGICIELS MAITRISES: WORD -- EXCEL -- SAGE -- POWERPOINT -- OUTLOOK',
  {'entities': [(21, 25, 'U-SKILLS'),
    (29, 34, 'U-SKILLS'),
    (38, 42, 'U-SKILLS'),
    (46, 56, 'U-SKILLS'),
    (60, 67, 'U-SKILLS')]}),
 ("COMPETENCES    VV VV VV VV VV V WV    Recevoir, orienter et renseigner les clients    Gerer les agendas, prendre les rendez-vous et traiter les appels telephoniques  Completer et mettre en forme des ecrits professionnels courants    Saisir, actualiser et controler des donnees    Etablir les contrats de travail, realiser les DPAE et programmer les visites medicales  Gerer les contrats, les bons de commande et les notes de frais de l'entreprise  Organiser les deplacements du personnel    Realiser les rapprochements bancaires    Gerer la facturation et le suivi de facturation    Organiser les reunions et les seminaires    Gerer les stocks et les fournitures de l'entreprise    OE Ks1-1 0011 alee lio] eM) Mee Realise y xo mel me (ele aal=lal es",
  {'entities': []}),
 ('Com

### Format Datas apres transformation --> (texte, listes de tuples)

In [16]:
list(set([tuple_[2] for tuple_ in DATA[0][1]['entities']]))

['U-SKILLS']

### Modeling -- Ci dessous 2 differents NER

In [19]:
def train_model_2(LABEL, TRAIN):
    # Setting up the pipeline and entity recognizer.
    nlp = spacy.blank('fr')  # create blank Language class
    print("Created blank 'fr' model")
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe('ner')
    else:
        ner = nlp.get_pipe('ner')
    # Add new entity labels to entity recognizer
    for i in LABEL:
        ner.add_label(i)
    # Inititalizing optimizer
    optimizer = nlp.begin_training()
    # Get names of other pipes to disable them during training to train # only NER and update the weights
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        for itn in range(40):
            random.shuffle(TRAIN)
            losses = {}
            batches = minibatch(TRAIN, 
                                size=compounding(4., 32., 1.001))
            for batch in batches:
                for text, annotations in batch:
                    # create Example
                    doc = nlp.make_doc(text)
                    example = Example.from_dict(doc, annotations)
                    # Update the model
                    nlp.update([example], losses=losses, drop=0.3)
    return nlp

In [10]:
LABEL_skills = ["U-SKILLS", "B-SKILLS", "I-SKILLS", "L-SKILLS", "U-ANGLAIS", "U-NIVEAU_ANGLAIS",
    "B-NIVEAU_ANGLAIS", "I-NIVEAU_ANGLAIS", "L-NIVEAU_ANGLAIS", "U-OTHER_LANGUAGE"]
LABEL_exp_pro = ['B-JobTitle','B-DateExp','I-DateExp','L-JobTitle','L-DateExp','I-JobTitle',
 'U-Company','L-Company','B-Company']

nlp = train_model_2(LABEL_skills, TRAIN)

Created blank 'fr' model


/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "Experiences professionnelles SAUTDELIGNE  SAUTDELI..." with entities "[(54, 64, 'B-JobTitle'), (65, 71, 'L-JobTitle'), (...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "aR Olga SORES SAUTDELIGNE  SAUTDELIGNE >>> Mai 202..." with entities "[(43, 51, 'B-DateExp'), (51, 54, 'I-DateExp'), (55...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/sp

/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "2016-2019 SAUTDELIGNE  SAUTDELIGNE 2017-2019 SAUTD..." with entities "[(0, 4, 'B-DateExp'), (4, 5, 'I-DateExp'), (5, 9, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "EXPERIENCES PROFESSIONNELLES SAUTDELIGNE  SAUTDELI..." with entities "[(54, 61, 'B-Company'), (62, 67, 'L-Company'), (92...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/sp

/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "EXPERIENCE PROFESSIONNELLE SAUTDELIGNE  SAUTDELIGN..." with entities "[(52, 55, 'B-Company'), (56, 62, 'I-Company'), (63...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "EXPERIENCE SAUTDELIGNE  SAUTDELIGNE ASSOCIATION VA..." with entities "[(460, 465, 'B-Company'), (465, 471, 'L-Company'),...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/sp

/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "Experiences professionnelles SAUTDELIGNE  SAUTDELI..." with entities "[(56, 68, 'B-DateExp'), (69, 70, 'I-DateExp'), (83...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "EXPERIENCE DANS LE DOMAINE DE L'ECRIT ET DE LA COR..." with entities "[(83, 92, 'B-DateExp'), (93, 95, 'I-DateExp'), (96...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/sp

...

/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "M Experiences SAUTDELIGNE  SAUTDELIGNE Depuis le 0..." with entities "[(49, 59, 'U-DateExp'), (80, 90, 'B-JobTitle'), (1...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "Aoit 2018 SAUTDELIGNE CDI SAUTDELIGNE  SAUTDELIGNE..." with entities "[(0, 9, 'U-DateExp'), (118, 127, 'B-DateExp'), (14...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/sp

/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "Experiences SAUTDELIGNE  SAUTDELIGNE Employeee de ..." with entities "[(37, 47, 'B-JobTitle'), (47, 49, 'I-JobTitle'), (...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "EXPERIENCES PROFESSIONNELLES SAUTDELIGNE  SAUTDELI..." with entities "[(601, 611, 'B-JobTitle'), (611, 614, 'I-JobTitle'...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/sp

/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "EXPERIENCES PROFESSIONNELLES SAUTDELIGNE  SAUTDELI..." with entities "[(54, 61, 'B-DateExp'), (62, 63, 'I-DateExp'), (63...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "Juillet 2019 -- Decembre 2019 (6 mois) SAUTDELIGNE..." with entities "[(0, 12, 'B-DateExp'), (13, 15, 'I-DateExp'), (15,...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/sp

/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "Mars SAUTDELIGNE 2019 SAUTDELIGNE  SAUTDELIGNE WED..." with entities "[(391, 400, 'U-JobTitle'), (401, 404, 'B-Company')...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "By isqacieness SAUTDELIGNE  SAUTDELIGNE   SAUTDELI..." with entities "[(132, 141, 'U-Company'), (67, 76, 'B-DateExp'), (...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/sp

/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "PARCOURS PROFESSIONNEL SAUTDELIGNE  SAUTDELIGNE   ..." with entities "[(75, 83, 'U-Company'), (99, 110, 'B-DateExp'), (1...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "EXPERIENCES PROFESSIONNELLES | SAUTDELIGNE  SAUTDE..." with entities "[(56, 64, 'B-Company'), (64, 76, 'I-Company'), (76...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/sp

### Save a model

In [12]:
def save_model(nlp, output_dir, model_name):
    """Input: nlp (model), directory ou l'on souhaite save le mpdel,
    model_name est le nom qu'on souhaite donner a notre modele"""
    # Save model 
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta['name'] = model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

save_model(nlp, "/Users/remyadda/Desktop/AD/Projets/ALFRED/ALFRED_PROJECT/Model_NER_3/Modeles", 'ner')

Saved model to /Users/remyadda/Desktop/AD/Projets/ALFRED/ALFRED_PROJECT/Model_NER_3/Modeles


### Test on new datas

In [30]:
def test_ner_model(output_dir, test_text):
    """Permet de tester un NER
    Input: directory du modele et texte a tester ainsi que le texte
    Output: Renvoie les labels predits ainsi que le texte correspondant a chaque label"""
    # Test the saved model
    nlp = spacy.load(output_dir)
    doc = nlp(test_text)
    predicted_labels = []
    corresponding_text = []
    for ent in doc.ents:
        predicted_labels.append(ent.label_)
        corresponding_text.append(ent.text)
    return predicted_labels, corresponding_text

In [31]:
TEST[0]

("Experience SAUTDELIGNE  SAUTDELIGNE Acheteuse SAUTDELIGNE CREDIT AGRICOLE LEASING & FACTORING - Montrouge (92) SAUTDELIGNE novembre 2018 - aout 2020 SAUTDELIGNE  SAUTDELIGNE C/ Outil achats (IVALUA) SAUTDELIGNE  SAUTDELIGNE C/<< Reporting SAUTDELIGNE  SAUTDELIGNE C/e Appels d'Offres SAUTDELIGNE  SAUTDELIGNE e Plenieres achats SAUTDELIGNE  SAUTDELIGNE C/<< Negociation des contrats SAUTDELIGNE  SAUTDELIGNE Chef de projet SAUTDELIGNE PRESTIGE AVENUE - Paris 8e (75) SAUTDELIGNE mars 2018 - aout 2018",
 {'entities': [(36, 45, 'U-JobTitle'),
   (57, 64, 'B-Company'),
   (65, 73, 'I-Company'),
   (74, 81, 'I-Company'),
   (82, 83, 'I-Company'),
   (84, 93, 'L-Company'),
   (123, 136, 'B-DateExp'),
   (137, 138, 'I-DateExp'),
   (138, 148, 'L-DateExp'),
   (408, 413, 'B-JobTitle'),
   (413, 415, 'I-JobTitle'),
   (416, 422, 'L-JobTitle'),
   (435, 443, 'B-Company'),
   (444, 450, 'L-Company'),
   (479, 488, 'B-DateExp'),
   (489, 491, 'I-DateExp'),
   (491, 500, 'L-DateExp')]})

In [32]:
def retrieve_trueLabelAndWord(DATASET):
    """Permet de recuperer deux listes: True label, et le word correspondant
    Ceci permettra de comparer les valeurs reelles aux valeurs predites du NER"""
    true_labels = []
    corresponding_text = []
    for element in DATASET:
        # on retrieve le texte
        text = element[0]
        label = element[1]
        label = label['entities']
        for e in label:
            start = e[0]
            end = e[1]
            target = e[2]
            true_labels.append(target)
            corresponding_word = text[start:end]
            corresponding_text.append(corresponding_word)
    return true_labels, corresponding_text

In [33]:
def X_test(DATASET):
    """Permet de recuperer le texte afin d'effectuer les predictions"""
    texts = []
    for element in DATASET:
        text = element[0]
        texts.append(text)
    return texts

In [43]:
Xtest = X_test(TEST)
print(Xtest[0])

EXPERIENCES PROFESSIONNELLES SAUTDELIGNE  SAUTDELIGNE Collaborateur Caisse SAUTDELIGNE  SAUTDELIGNE Chanel SAUTDELIGNE  SAUTDELIGNE 2018 & 2020, Paris SAUTDELIGNE  SAUTDELIGNE = Accompagnement du client dans son parcours d'achat SAUTDELIGNE =" Ventes additionnelles d'articles en caisse SAUTDELIGNE  SAUTDELIGNE Agent de maitrise Caisses & CRM SAUTDELIGNE  SAUTDELIGNE Prada SAUTDELIGNE  SAUTDELIGNE yA Os EA 0N Re rela SAUTDELIGNE  SAUTDELIGNE = Responsable des flux de caisse et des services SAUTDELIGNE generaux de la boutique (fournitures, contrats de SAUTDELIGNE maintenance, entretien, uniformes, R.H.) SAUTDELIGNE  SAUTDELIGNE =" Enrichissements des donnees CRM SAUTDELIGNE  SAUTDELIGNE = Management en boutique SAUTDELIGNE  SAUTDELIGNE Collaborateur Caisse SAUTDELIGNE  SAUTDELIGNE Gucci Montaigne SAUTDELIGNE  SAUTDELIGNE yA ON TC SAUTDELIGNE  SAUTDELIGNE =" Creation d'outils de pilotage (Tableaux Excel) SAUTDELIGNE  SAUTDELIGNE Conseiller Vendeur & SAV SAUTDELIGNE  SAUTDELIGNE aA ULI SAU

#### 1- Récupération des tokens et labels que le modèle devrait être capable de prédire (True tuples on les appelera)

In [45]:
true_labels, true_corresponding_words = retrieve_trueLabelAndWord(TEST)
print('True labels : ', true_labels[0], ' --> True corresponding word : ',true_corresponding_words[0])
# Create list of tuples (word, label_predicted) --> useful for metrics

def make_tuples(true_token, true_labels):
    """Input: la liste de mots (token) ainsi que la liste de labels correspondante
       Output: liste de tuples ou chaque tuple est de la forme (token, label)"""
    true_tuple = []
    for w, p in zip(true_token, true_labels):
        true_tuple.append((w, p))

    print(len(true_tuple), 'tuples')
    return true_tuple

# Nbe de tuples a trouver
true_tuple = make_tuples(true_corresponding_words, true_labels)

True labels :  B-JobTitle  --> True corresponding word :  Collaborateur
8890 tuples


### Predictions

In [38]:
# On predit chaque texte de Xtest
output_dir = "/Users/remyadda/Desktop/AD/Projets/ALFRED/ALFRED_PROJECT/Model_NER_3"
preds = []
predicted_corresponding_word_list = []
for text in Xtest:
    predictions, predicted_corresponding_word = test_ner_model(output_dir, text)
    preds.extend(predictions)
    predicted_corresponding_word_list.extend(predicted_corresponding_word)

#### 2 - Récupération des tokens et labels que le modèle a prédit (predicted tuples on les appelera)

In [39]:
# Create list of tuples (word, label_predicted) --> useful for metrics
# Nbe de tuples prédits
predicted_tuple = make_tuples(predicted_corresponding_word_list, preds)

1507 tuples


### Defintion des Metrics

In [41]:
# Explication : supposons que dans le Test set il y ait 200 mots a labeliser, et que le modele en trouve 100, 
# alors, le coverage serait de 50%. Certes toutes les predictions peuvent etre bonnes, mais il en manque la moitié.
def get_metrics(y_pred, y_true):
    """Input: y_pred et y_true sous forme de listes de tuple (texte, label)
    Renvoie le poucentage de bons tuples (texte, label) détectés par le modele"""
    y_true_copy = y_true.copy()
    y_pred_copy = y_pred.copy()
    wrong_tuples = []
    good_tuples = []
    count = 0
    for tuple_ in y_pred_copy:
        count += 1
        if tuple_ in y_true_copy:
            y_true_copy.remove(tuple_)
            good_tuples.append(tuple_)
        else:
            # tuple qui na pas lieu d'exister
            wrong_tuples.append(tuple_)
    #print(count)
    accuracy_1 = np.round(len(good_tuples)/len(y_pred), 3) # cb de bonnes predictions par rap a ce que je predis
    accuracy_2 = np.round(len(good_tuples)/len(y_true), 3) # % cb de bonnes predictions par rap a ce quil fallait trouver
    
    #lets take a look on bad predictions
    # erreur de type 1 : mot trouvé, mais prédiction fausse
    label_error = {} # on spotera les labels où le modèle fait erreur
    count_error_I = 0
    for tuple_ in wrong_tuples:
        for TrueTuple in y_true_copy:
            if tuple_[0] == TrueTuple[0]:
                count_error_I += 1
                if (tuple_[1], TrueTuple[1]) not in label_error:
                    label_error[(tuple_[1], TrueTuple[1])] = 1
                else:
                    label_error[(tuple_[1], TrueTuple[1])] += 1
                y_true_copy.remove(TrueTuple)
                break
        # Si c'est pas une erreur de type I c'est qu'il s'agit d'une erreur de type II 
        # cad un mot releve qui n'aurait jamais du l'etre
    count_error_II = len(wrong_tuples) - count_error_I
    coverage = np.round(len(y_true) / len(y_pred) * 100, 1)
    coverage = np.round(np.abs(100 - coverage), 1)
    print("Nombre de mots à prédire pour l'ensemble du test set : ", len(y_true))
    print("Nombre de mots prédits par le NER pour l'ensemble du test set : ", len(y_pred))
    print('Coverage : ', coverage, '% de difference')
    print('----------------------------------------------------------------------')
    print(len(good_tuples), ' mots ont bien été relevés ET prédits par le NER')
    print(len(wrong_tuples), " mots ont mal été prédits (erreur de type I ou type II)")
    print('Bonnes prédictions + Mauvaises prédictions =', len(good_tuples)+len(wrong_tuples))
    print('----------------------------------------------------------------------')
    print('Accuracy I (bonne pred / total pred) : ', accuracy_1*100, ' %')
    print('Accuracy II (bonne pred / total expected pred) : ', accuracy_2*100, ' %')
    print('----------------------------------------------------------------------')
    print('Nbe erreur type I : ', count_error_I, ' (mots bien relevés mais mal prédits)')
    print('Nbe erreur type II : ', count_error_II, " (mots qui n'auraient pas du être prédits)")
    print('----------------------------------------------------------------------')
    print("Confusions faites par l'algo (prediction, true_label): \n", label_error)

#### Lets check our results

In [42]:
get_metrics(predicted_tuple, true_tuple)

Nombre de mots à prédire pour l'ensemble du test set :  1772
Nombre de mots prédits par le NER pour l'ensemble du test set :  1507
Coverage :  17.6 % de difference
----------------------------------------------------------------------
852  mots ont bien été relevés ET prédits par le NER
655  mots ont mal été prédits (erreur de type I ou type II)
Bonnes prédictions + Mauvaises prédictions = 1507
----------------------------------------------------------------------
Accuracy I (bonne pred / total pred) :  56.49999999999999  %
Accuracy II (bonne pred / total expected pred) :  48.1  %
----------------------------------------------------------------------
Nbe erreur type I :  95  (mots bien relevés mais mal prédits)
Nbe erreur type II :  560  (mots qui n'auraient pas du être prédits)
----------------------------------------------------------------------
Confusions faites par l'algo (prediction, true_label): 
 {('L-JobTitle', 'I-JobTitle'): 13, ('B-JobTitle', 'I-JobTitle'): 2, ('L-Company', 

#### Pour appliquer cette fonction metric, il faut d'abord diviser votre DATASET en TRAIN et TEST.
    Puis, dans le cas ou votre NER prend ceci en INPUT : 

    INPUT :
    ('LOGICIELS MAITRISES: WORD -- EXCEL -- SAGE -- POWERPOINT -- OUTLOOK',
     {'entities': [(21, 25, 'U-SKILLS'),
       (29, 34, 'U-SKILLS'),
       (38, 42, 'U-SKILLS'),
       (46, 56, 'U-SKILLS'),
       (60, 67, 'U-SKILLS')]}), 
   
    entrainez votre NER sur le train set.
    
    Afin de tester ses performances sur le test, recuperer les mots ainsi que les preditions qu'il aurait du       pouvoir effectuer dans le meilleur des cas, pour cela, utilisez la fonction "retrieve_trueLabelAndWord" qui prend en input l'INPUT ci dessus, et renvoie deux listes : true label et mots à predire.
    
        Utiliser ensuite la fonction "make_tuple" afin de merge ces deux listes en une seule liste de tuples.
        On utilise deux fois cette fonction : 
        - pour les reels labels a trouver
        - pour les prediction effectuées par l'algo
        
        Enfin on applique la fonction get_metrics
    